## **Elaborazione di Immagini Mediche**
### Contest 2021/22 - Segmentazione ghiandola prostatica in immagini MRI



***
***- - - Collegamento a Google Drive e import delle e librerie files necessari - - -***
***


In [8]:
# Before running the script, reset the runtime to factory reset (Runtime -> Factory Reset Runtime)
# and then change runtime type to GPU (Runtime -> Change runtime type)

# Install libriary dependencies for running deep learning
!pip install tensorflow==2.1.0
!pip install keras==2.3.1
!pip install segmentation_models==1.0.1
!pip install h5py==2.10.0 
!pip install plotly==5.3.1

In [11]:
from google.colab import drive
drive.mount('/content/drive')

import os
import random
import numpy as np
import plotly.express as px
import cv2
import scipy

from matplotlib import pyplot as plt
from tqdm import tqdm
from skimage.io import imread, imshow, imsave
from skimage.transform import resize
from skimage.segmentation import mark_boundaries
from scipy import ndimage
from skimage.util import img_as_ubyte, crop
from skimage.morphology import binary_dilation,remove_small_objects

from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

from segmentation_models import Unet

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# LOADING DATASET IN .rar FORMAT FROM DRIVE TO COLAB AND CREATING TESTING DIRECTORIES

# loading dataset
!pip install unrar
!unrar x "drive/MyDrive/gruppo_FA_DO_PA_PA/DATASET_stu.rar"   # inserire path completa in cui si trovano i dati (es. "drive/MyDrive/..../DATASET_stu.rar")

# creating directories in which to store the results

dataset_name = "DATASET_stu"

path_results_tr = os.path.join(dataset_name,"train","automatic")
if not os.path.exists(path_results_tr):
  os.mkdir(path_results_tr)
  
path_results_vl = os.path.join(dataset_name,"val","automatic")
if not os.path.exists(path_results_vl):
  os.mkdir(path_results_vl)

path_results_ts = os.path.join(dataset_name,"test","automatic")
if not os.path.exists(path_results_ts):
  os.mkdir(path_results_ts)


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from drive/MyDrive/gruppo_FA_DO_PA_PA/DATASET_stu.rar

Creating    DATASET_stu                                               OK
Creating    DATASET_stu/test                                          OK
Creating    DATASET_stu/test/images                                   OK
Extracting  DATASET_stu/test/images/2001.tiff                              1%  2%  OK 
Extracting  DATASET_stu/test/images/2022.tiff                              4%  OK 
Extracting  DATASET_stu/test/images/3032.tiff                              6%  OK 
Extracting  DATASET_stu/test/images/3068.tiff                              7%  8%  OK 
Extracting  DATASET_stu/test/images/3268.tiff                              9%  OK 
Extracting  DATASET_stu/test/images/MIP-PROSTATE-01-0002.tiff             11%  OK 
Extracting  DATASET_stu/test/images/MIP-PROSTATE-01-0008.tiff             

***
***- - - TESTING - - -***
***

In [4]:
#Loading trained model (please wait it will take nearly 4 minutes to load 3 densenet trained models)

current_net1 = 'x1'
current_net2 = 'x2'
current_net3 = 'x3'
model1 = load_model('drive/MyDrive/gruppo_FA_DO_PA_PA/_TRAINED_MODELS/' + current_net1) 
model2 = load_model('drive/MyDrive/gruppo_FA_DO_PA_PA/_TRAINED_MODELS/' + current_net2)   
model3 = load_model('drive/MyDrive/gruppo_FA_DO_PA_PA/_TRAINED_MODELS/' + current_net3)

models = [model1,model2,model3]

In [23]:
# PREDICTING AND CALCULATING PERFORMANCES ON TRAINING SET 

# Path
TRAIN_IMG_path = os.path.join(dataset_name,'train','images')  # path to training volumes of images 
TRAIN_MASK_path = os.path.join(dataset_name,'train','manual')  # path respective masks 

# Feeding training examples to trained UNET
train_images = os.listdir(TRAIN_IMG_path)   #list of training examples
perf_train = np.full((len(train_images),3), np.nan)   # array to store DSC on each volume (we calculated DSC on volumes and not on slices beacuse the latter does not
                                                      # discriminate the entity of the fails of the model when TP are equal to 0 on these slices)
for n, id_ in tqdm(enumerate(train_images), total=len(train_images)):
    
    # Load and prepocess RM images
    vol = imread(TRAIN_IMG_path+'/'+id_)  #importing volume in format [# slices,slice_height,slice_width]
    vol = crop(vol,((0,0),(128,128),(100,156)),copy=False)   #cropping  
    for sl in range(vol.shape[0]):
       vol[sl,:,:] = ndimage.median_filter(vol[sl,:,:],3)   #filtering w/ median filter of kernel size 3x3     

    # Load manual mask
    mask_manu = imread(TRAIN_MASK_path+'/'+id_)   
    
    # Apply UNETs for prediction (ensembling at probability maps level is done)
    softmax = np.zeros((vol.shape[0],vol.shape[1],vol.shape[2],2),dtype=np.float32)
    for i in range(len(models)):
        softmax =softmax + models[i].predict(np.stack((vol,vol,vol),axis=3))   #conversion from grayscale to rgb is done here (on fly, w/ stack routine) for each volume
    softmax[softmax < 1.05] = 0
    softmax[softmax >= 1.05] = 1
    mask_auto = (np.copy(softmax[:,:,:,1])).astype(np.uint8)
    mask_auto = np.pad(mask_auto,((0,0),(128,128),(100,156)),'constant', constant_values=0) #uncropping (padding w/ 0s) of the obtained mask
    
    # post processing
    where_pred = np.any(mask_auto,axis=(1,2)) # array indicating slices on which we predicted prostate
    where_pred = scipy.ndimage.distance_transform_edt(where_pred)
    for sl in range(mask_auto.shape[0]):
        
        mask_auto[sl] = remove_small_objects(mask_auto[sl].astype(np.bool), min_size=1600, connectivity=2) #remove small objects
        kernel  = np.ones((3, 3), np.uint8) #kernel for dilation
        mask_auto[sl] = cv2.dilate(mask_auto[sl],kernel) #dilation 
        labeled, num_trees = scipy.ndimage.label(~(mask_auto[sl].astype(np.bool))) #check how many black holes there are in predicted mask after dilation 
        if sl != 0 and sl != (mask_auto.shape[0]-1):
            if where_pred[sl] ==1 and num_trees > 1:  # if we are at the border of predicted prostate and there are holes in predicted mask of slices
                mask_auto[sl,:,:] = 0
        mask_auto[sl] = cv2.erode(mask_auto[sl],kernel,iterations=1) #erosion after dilation

        #Morph close (to remove small holes in predicted mask considered as true masks)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))  #kernel for closure
        mask_auto[sl]  = cv2.morphologyEx(mask_auto[sl] , cv2.MORPH_CLOSE, kernel, iterations=10)
      
        #smoothing contours 
        scale_up  = cv2.pyrUp(mask_auto[sl])
        scale_up = cv2.medianBlur(scale_up,5)
        scale_down = cv2.pyrDown(scale_up)
        mask_auto[sl] = scale_down 


    imsave(os.path.join(path_results_tr,id_),mask_auto,check_contrast=False)

    #evaluating performances on current volume
    TP = (mask_auto.astype(bool)*mask_manu.astype(bool)).sum()
    FP = (mask_auto.astype(bool)*~mask_manu.astype(bool)).sum()        
    FN = (~mask_auto.astype(bool)*mask_manu.astype(bool)).sum()       
    perf_train[n,0] = TP/(TP+FP)             #precision on n-th volume
    perf_train[n,1] = TP/(TP+FN)             #recall on  n-th volume
    perf_train[n,2] = 2*TP/(2*TP+FP+FN)     #F-SCORE on n-th volume




#PREDICTING AND CALCULATING PERFORMANCES ON VALIDATION SET

# Path
VAL_IMG_path = os.path.join(dataset_name,'val','images')  # path to test images 
VAL_MASK_path = os.path.join(dataset_name,'val','manual')  # path to masks

# Feeding validation examples to trained UNET
val_images = os.listdir(VAL_IMG_path)
perf_val = np.full((len(val_images),3), np.nan)
for n, id_ in tqdm(enumerate(val_images), total=len(val_images)):
    
    # Load and prepocess RM images
    vol = imread(VAL_IMG_path+'/'+id_)  #importing volume in format [# slices,slice_height,slice_width] 
    vol = crop(vol,((0,0),(128,128),(100,156)),copy=False) #cropping to 256x256 
    for sl in range(vol.shape[0]):
       vol[sl,:,:] = ndimage.median_filter(vol[sl,:,:],3)   #filtering w/ median filter of kernel size 3x3 

    # Load manual mask
    mask_manu = imread(VAL_MASK_path+'/'+id_)   
    
    # Apply UNETs for prediction (ensembling at probability maps level is done)
    softmax = np.zeros((vol.shape[0],vol.shape[1],vol.shape[2],2),dtype=np.float32)
    for i in range(len(models)):
        softmax =softmax + models[i].predict(np.stack((vol,vol,vol),axis=3))   #conversion from grayscale to rgb is done here (on fly, w/ stack routine) for each volume
    softmax[softmax < 1.05] = 0
    softmax[softmax >= 1.05] = 1
    mask_auto = (np.copy(softmax[:,:,:,1])).astype(np.uint8)
    mask_auto = np.pad(mask_auto,((0,0),(128,128),(100,156)),'constant', constant_values=0) #uncropping (padding w/ 0s) of the obtained mask

    # post processing
    where_pred = np.any(mask_auto,axis=(1,2)) # array indicating slices on which we predicted prostate
    where_pred = scipy.ndimage.distance_transform_edt(where_pred)
    for sl in range(mask_auto.shape[0]):
        
        mask_auto[sl] = remove_small_objects(mask_auto[sl].astype(np.bool), min_size=1600, connectivity=2) #remove small objects
        kernel  = np.ones((3, 3), np.uint8) #kernel for dilation
        mask_auto[sl] = cv2.dilate(mask_auto[sl],kernel) #dilation 
        labeled, num_trees = scipy.ndimage.label(~(mask_auto[sl].astype(np.bool))) #check how many black holes there are in predicted mask after dilation 
        if sl != 0 and sl != (mask_auto.shape[0]-1):
            if where_pred[sl] ==1 and num_trees > 1:  # if we are at the border of predicted prostate and there are holes in predicted mask of slices
                mask_auto[sl,:,:] = 0
        mask_auto[sl] = cv2.erode(mask_auto[sl],kernel,iterations=1) #erosion after dilation

        #Morph close (to remove small holes in predicted mask considered as true masks)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))  #kernel for closure
        mask_auto[sl]  = cv2.morphologyEx(mask_auto[sl] , cv2.MORPH_CLOSE, kernel, iterations=10)
      
        #smoothing contours 
        scale_up  = cv2.pyrUp(mask_auto[sl])
        scale_up = cv2.medianBlur(scale_up,5)
        scale_down = cv2.pyrDown(scale_up)
        mask_auto[sl] = scale_down 


    imsave(os.path.join(path_results_vl,id_),mask_auto,check_contrast=False)

    #evaluating performances
    TP = (mask_auto.astype(bool)*mask_manu.astype(bool)).sum()
    FP = (mask_auto.astype(bool)*~mask_manu.astype(bool)).sum()        
    FN = (~mask_auto.astype(bool)*mask_manu.astype(bool)).sum()     
    perf_val[n,0] = TP/(TP+FP)             #precision on n-th volume
    perf_val[n,1] = TP/(TP+FN)             #recall on  n-th volume
    perf_val[n,2] = 2*TP/(2*TP+FP+FN)     #F-SCORE on n-th volume



# PREDICTING ON TEST SET

# Path
TEST_IMG_path = os.path.join(dataset_name,'test','images')  # path to test examples 

# Feeding validation examples to trained UNET
test_images = os.listdir(TEST_IMG_path)
for n, id_ in tqdm(enumerate(test_images), total=len(test_images)):
    
    vol = imread(TEST_IMG_path+'/'+id_)
    vol = crop(vol,((0,0),(128,128),(100,156)),copy=False) # cropping images to 256x256
    
    for sl in range(vol.shape[0]):
       vol[sl,:,:] = ndimage.median_filter(vol[sl,:,:],3)
    
    # Apply UNETs for prediction (ensembling at probability maps level is done)
    softmax = np.zeros((vol.shape[0],vol.shape[1],vol.shape[2],2),dtype=np.float32)
    for i in range(len(models)):
        softmax =softmax + models[i].predict(np.stack((vol,vol,vol),axis=3))   #conversion from grayscale to rgb is done here (on fly, w/ stack routine) for each volume
    softmax[softmax < 1.05] = 0
    softmax[softmax >= 1.05] = 1
    mask_auto = (np.copy(softmax[:,:,:,1])).astype(np.uint8)
    mask_auto = np.pad(mask_auto,((0,0),(128,128),(100,156)),'constant', constant_values=0) #uncropping (padding w/ 0s) of the obtained mask

    # post processing
    where_pred = np.any(mask_auto,axis=(1,2)) # array indicating slices on which we predicted prostate
    where_pred = scipy.ndimage.distance_transform_edt(where_pred)
    for sl in range(mask_auto.shape[0]):
        
        mask_auto[sl] = remove_small_objects(mask_auto[sl].astype(np.bool), min_size=1600, connectivity=2) #remove small objects
        kernel  = np.ones((3, 3), np.uint8) #kernel for dilation
        mask_auto[sl] = cv2.dilate(mask_auto[sl],kernel) #dilation 
        labeled, num_trees = scipy.ndimage.label(~(mask_auto[sl].astype(np.bool))) #check how many black holes there are in predicted mask after dilation 
        if sl != 0 and sl != (mask_auto.shape[0]-1):
            if where_pred[sl] == 1 and num_trees > 1:  # if we are at the border of predicted prostate and there are holes in predicted mask of slices
                mask_auto[sl,:,:] = 0
        mask_auto[sl] = cv2.erode(mask_auto[sl],kernel,iterations=1) #erosion after dilation

        #Morph close (to remove small holes in predicted mask considered as true masks)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))  #kernel for closure
        mask_auto[sl]  = cv2.morphologyEx(mask_auto[sl] , cv2.MORPH_CLOSE, kernel, iterations=10)
      
        #smoothing contours 
        scale_up  = cv2.pyrUp(mask_auto[sl])
        scale_up = cv2.medianBlur(scale_up,5)
        scale_down = cv2.pyrDown(scale_up)
        mask_auto[sl] = scale_down 
        
    
    imsave(os.path.join(path_results_ts,id_),mask_auto,check_contrast=False)

#PERFORMANCES PRINT
overall_tr = np.mean(perf_train,axis=0)
overall_val = np.mean(perf_val,axis=0)

print(f"\n\nperformances on TRAINING SET\n\t- precision: {overall_tr[0]}\n\t- recall: {overall_tr[1]}\n\t- F-SCORE: {overall_tr[2]}")
print(f"\n\nperformances on VALIDATION SET\n\t- precision: {overall_val[0]}\n\t- recall: {overall_val[1]}\n\t- F-SCORE: {overall_val[2]}")

100%|██████████| 10/10 [00:30<00:00,  3.04s/it]



performances on TRAINING SET
	- precision: 0.9443170067308986
	- recall: 0.9709378895610331
	- F-SCORE: 0.9573368047523169


performances on VALIDATION SET
	- precision: 0.9137620402873614
	- recall: 0.9365991104785185
	- F-SCORE: 0.923970868410057


In [9]:
# SAVING COMPRESSED RESULTS FROM COLAB TO DRIVE AS .rar

#compressing in .rar format 
!apt-get install rar
!rar a "DATASET_stu" "DATASET_stu"

#copy from Colab to drive (it will replace)
import shutil
shutil.copyfile(dataset_name + ".rar", "/content/drive/MyDrive/gruppo_FA_DO_PA_PA/DATASET_stu.rar")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
rar is already the newest version (2:5.5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.

RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help

Evaluation copy. Please register.

Updating archive DATASET_stu.rar

Updating  DATASET_stu/val/images/3154.tiff                                 0%  OK 
Updating  DATASET_stu/val/images/MIP-PROSTATE-01-0023.tiff                 1%  OK 
Updating  DATASET_stu/val/images/MIP-PROSTATE-01-0004.tiff                 1%  2%  OK 
Updating  DATASET_stu/val/images/MIP-PROSTATE-01-0017.tiff                 2%  OK 
Updating  DATASET_stu/val/images/MIP-PROSTATE-01-0020.tiff                 3%  OK 
Updating  DATASET_stu/val/images/3160.tiff                                 4%  OK 
Updating  DATASET_stu/val/images/2010.tiff     

'/content/drive/MyDrive/gruppo_FA_DO_PA_PA/DATASET_stu.rar'